In [2]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matrixprofile.discover import snippets
import pandas as pd
from pyts.image import RecurrencePlot
import tensorflow as tf
import json
from sklearn.model_selection import train_test_split
from scipy.ndimage.interpolation import zoom


In [3]:
def from_np_array(array_string):
    array_string = ','.join(array_string.replace('[ ', '[').split())
    return np.array(ast.literal_eval(array_string))

In [4]:
p = Path('../Dataset/WalkRun1_80_3800_200.csv')

df = pd.read_csv(p,converters={"neighbors":json.loads})

In [5]:
df

,Unnamed: 0,key,snippet,fraction,neighbors
0,0,0,[0.55714116 0.56319213 0.56522101 0.57919439 0...,0.311884,"[[0.014993, -0.043289, -0.0039098, -0.038406, ..."
1,1,1,[0.47102721 0.44296578 0.4652721 0.50352454 0...,0.430870,"[[7.0488, 4.9723, 4.6134, 3.3802, 2.4563, 2.16..."
2,2,2,[0.54415993 0.545161 0.54891279 0.55129218 0...,0.257536,"[[-0.038406, 0.001217, 0.0039053, -0.0056399, ..."


In [6]:

rp = RecurrencePlot(threshold='point')
X_rp = rp.fit_transform(np.array(df.neighbors.iloc[0][0]).reshape(1,-1))
X=[]
y=[]
for i,item in df.iterrows():
    for j in item.neighbors:
        if rp.transform(np.array(j).reshape(1,-1))[0].shape == (300,300):
            buf = zoom(rp.transform(np.array(j).reshape(1,-1))[0], 0.25)
            X.append(buf)
            y.append(item["key"])
X=np.stack(np.array(X))
y=np.array(y)

In [7]:
X.shape

(6703, 75, 75)

In [8]:
y = tf.keras.utils.to_categorical(y)

In [9]:
del X_train
del X_test
del X_valid

NameError: name 'X_train' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [ ]:
del X
del y

In [ ]:
from keras.layers import TimeDistributed
from keras.layers import Reshape
from keras.layers import Conv2D,Lambda, Activation, LSTM
from keras.utils import to_categorical
from keras.models import Model,Input
import keras 
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Reshape,GlobalAveragePooling2D,AveragePooling2D
from keras import backend as K
from tensorflow.keras.layers import GRU

In [ ]:
X_train.shape

In [ ]:
def model():
    model = Sequential()
    input_layer = Input((X_train.shape[2], X_train.shape[2], 1),name="img_input",dtype='float32')
    output = Conv2D(128, (5, 5), kernel_initializer='he_normal',  activation='relu')(input_layer)
    output = AveragePooling2D(pool_size=(2, 2))(output)
    output = Conv2D(128, (5, 5), kernel_initializer='he_normal',  activation='relu')(output)
    output = AveragePooling2D(pool_size=(2, 2))(output)
    output = Conv2D(64, (5, 5), kernel_initializer='he_normal', activation='relu')(output)
    output = GlobalAveragePooling2D()(output)
    output = Dropout(0.25)(output)
    output = Dense(y_train.shape[1])(output)
    y_pred = Activation('softmax', name='softmax')(output)
    
    
    model = Model(inputs=input_layer, outputs=y_pred)
    model.summary() 
    model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=[tf.keras.metrics.Accuracy()])
    
    return model

model = model()

In [ ]:
histor = model.fit(X_train,
          y_train,
          validation_data=(X_valid,y_valid),
          #callbacks=[learning_rate_reduction], 
          batch_size=25, epochs=60)

In [ ]:
pred= model.predict(X_test)
np.argmax(pred,axis=-1)
y_test_arg = np.argmax(y_test,axis=-1)
j = 0
for i, item in enumerate(np.argmax(pred,axis=-1)):
    if item!=y_test_arg[i]:
        j+=1
print((len(pred)-j)/len(pred))
plt.plot(histor.history["loss"])
plt.plot(histor.history["val_loss"])